In [18]:
from rouge_score import rouge_scorer
import matplotlib.pyplot as plt

In [17]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=79dbc4d6e7fa73ea7b7067dea7875c39462fdbeba1704d09d97acfb95094443c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [1]:
!pip install transformers datasets torch nltk

import json
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
import nltk
from nltk.tokenize import word_tokenize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [19]:
def load_tweets(file_path):
    tweets = []
    with open(file_path, 'r') as f:
        for line in f:
            try:
                tweet = json.loads(line)
                if 'text' in tweet:
                    tweets.append(tweet['text'])
            except json.JSONDecodeError:
                continue
    return tweets

file_path = '/content/corona.json'
tweets = load_tweets(file_path)
print(f"Loaded {len(tweets)} tweets.")


def preprocess_tweets(tweets):
    processed_tweets = []
    for tweet in tweets:
        tokens = word_tokenize(tweet)
        processed_tweets.append(" ".join(tokens))
    return processed_tweets

processed_tweets = preprocess_tweets(tweets)


tweet_dataset = Dataset.from_dict({"text": processed_tweets})


model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128, return_tensors="pt")
    inputs["labels"] = inputs["input_ids"].clone()
    return inputs

tokenized_dataset = tweet_dataset.map(tokenize_function, batched=True)



model = GPT2LMHeadModel.from_pretrained(model_name)


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    fp16=True if torch.cuda.is_available() else False,
)

def compute_metrics(eval_pred):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


    rouge_scores = []
    for pred, label in zip(decoded_preds, decoded_labels):
        scores = scorer.score(label, pred)
        rouge_scores.append(scores)


    avg_rouge1 = sum([s["rouge1"].fmeasure for s in rouge_scores]) / len(rouge_scores)
    avg_rouge2 = sum([s["rouge2"].fmeasure for s in rouge_scores]) / len(rouge_scores)
    avg_rougeL = sum([s["rougeL"].fmeasure for s in rouge_scores]) / len(rouge_scores)

    return {"rouge1": avg_rouge1, "rouge2": avg_rouge2, "rougeL": avg_rougeL}


# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
train_results =trainer.train()





Loaded 18518 tweets.


Map:   0%|          | 0/18518 [00:00<?, ? examples/s]

<ipython-input-19-966a2656ecb0>:79: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,1.608900
1000,1.293100
1500,1.199300
2000,1.137100
2500,1.107500
3000,0.980600
3500,0.998300
4000,0.977100
4500,0.967700
5000,0.906400


In [20]:
def generate_tweet(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)  # Move inputs to the model's device
    outputs = model.generate(inputs.input_ids, max_length=max_length, num_return_sequences=1, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [13]:
#1 epoch
prompt = "Corona"
generated_tweet = generate_tweet(prompt, model, tokenizer)
print(f"Generated Tweet: {generated_tweet}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Tweet: Corona is a virus .


In [34]:
#5 epoch
prompt = "Corona"
generated_tweet = generate_tweet(prompt, model, tokenizer)
print(f"Generated Tweet: {generated_tweet}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Tweet: Corona virus is a virus of the world . It is a virus of the world . It is a virus of the world . It is a virus of the world . It is a virus of the world .


In [ ]:
plt.plot(train_results.history["loss"], label="Training Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Training Loss Curve")
plt.legend()
plt.show()

In [39]:
def generate_tweet(prompt, model, tokenizer, max_length=50):
  """Generates a tweet based on the given prompt."""
  inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  outputs = model.generate(inputs.input_ids, max_length=max_length, num_return_sequences=1, temperature=0.7)
  return tokenizer.decode(outputs[0], skip_special_tokens=True)

def calculate_rouge(generated_text, reference_text):
  """Calculates ROUGE scores for generated and reference texts."""
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
  scores = scorer.score(reference_text[0]['text'], generated_text)
  return {metric: scores[metric].fmeasure for metric in scores}

# Example usage:
prompt = "Corona"
generated_tweet = generate_tweet(prompt, model, tokenizer)
print(f"Generated Tweet: {generated_tweet}")


# Calculate ROUGE scores
rouge_scores = calculate_rouge(generated_tweet, tokenized_dataset)

# Print the ROUGE scores
print(f"ROUGE-1: {rouge_scores['rouge1']:.4f}")
print(f"ROUGE-2: {rouge_scores['rouge2']:.4f}")
print(f"ROUGE-L: {rouge_scores['rougeL']:.4f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Tweet: Corona virus is a virus that can be spread through kissing . https : //t.co/Xn0CqxnQ7
ROUGE-1: 0.0833
ROUGE-2: 0.0000
ROUGE-L: 0.0833


In [41]:
from rouge_score import rouge_scorer

def generate_tweet(prompt, model, tokenizer, max_length=50):
  """Generates a tweet based on the given prompt."""
  inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  outputs = model.generate(inputs.input_ids, max_length=max_length, num_return_sequences=1, temperature=0.7)
  return tokenizer.decode(outputs[0], skip_special_tokens=True)

def calculate_rouge(generated_text, reference_texts):
  """Calculates average ROUGE scores for generated text against a list of reference texts."""
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
  total_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
  num_references = len(reference_texts)

  for reference_text in reference_texts:
      scores = scorer.score(reference_text, generated_text)
      for metric in scores:
          total_scores[metric] += scores[metric].fmeasure

  # Calculate average scores
  avg_scores = {metric: total_scores[metric] / num_references for metric in total_scores}
  return avg_scores

# Example usage:
prompt = "Corona"
generated_tweet = generate_tweet(prompt, model, tokenizer)
print(f"Generated Tweet: {generated_tweet}")


rouge_scores = calculate_rouge(generated_tweet, tokenized_dataset['text'])

# Print the average ROUGE scores
print(f"Average ROUGE-1: {rouge_scores['rouge1']:.4f}")
print(f"Average ROUGE-2: {rouge_scores['rouge2']:.4f}")
print(f"Average ROUGE-L: {rouge_scores['rougeL']:.4f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Tweet: Corona-Krise : The New York Times bestsellers : Coronavirus , The New York Times bestsellers : The Invisible Enemy , The New York Times bestsellers… https : //t.co/0Q0c
Average ROUGE-1: 0.0890
Average ROUGE-2: 0.0302
Average ROUGE-L: 0.0847
